In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(appName="YourTest", master="local[*]")

In [ ]:
from spamminess import spamminess
from math import exp
import shutil, os

def spark_SGD(training_dataset='spam.train.group_x.txt', output_model='models/group_x_model', delta = 0.002):
    def spammi(x):
      model = {}
      for l in x:
        t = l[0]
        F = l[1]
        score = spamminess(F,model)
        prob = 1.0/(1+exp(-score))
        for f in F:
          if t == 'spam':
            if f in model:
              model[f] = model[f] + (1.0-prob)*delta
            else:
              model[f] = (1.0-prob)*delta
          elif t == 'ham':
            if f in model:
              model[f] = model[f] - prob*delta
            else:
              model[f] = -prob*delta
      for (f,w) in model.items():
        yield((int(f),w))


    if os.path.isdir(output_model):
        shutil.rmtree(output_model) # Remove the previous model to create a new one
    training_data = sc.textFile(training_dataset)
    data = training_data.map(lambda x : x.split(" ")).map(lambda x:(x[1],x[2:])).coalesce(1)
    train = data.mapPartitions(spammi)
    train.saveAsTextFile(output_model)

In [ ]:
def spark_classify(input_model='models/group_x_model', test_dataset='spam.test.qrels.txt', results_path='results/test_qrels'):
    if os.path.isdir(results_path):
        shutil.rmtree(results_path) # Remove the previous results
    test_data = sc.textFile(test_dataset)
    d = {}
    with open(input_model+"/part-00000") as f:
      for line in f:
        (key, val) = line.strip().strip('()').split(",")
        d[key] = float(val)
    data = test_data.map(lambda x : x.split(" ")).map(lambda x:(x[0],x[1],x[2:]))
    test = data.map(lambda x : (x[0],x[1],spamminess(x[2],d))).map(lambda x : (x[0],x[1],x[2],("spam" if x[2] > 0 else "ham")))
    test.saveAsTextFile(results_path)